In [1]:
from scripts.models import TransformerModel
model = TransformerModel(
    n_dims=20,
    n_positions=101,
    n_embd=256,
    n_layer=12,
    n_head=8,
    pred_type="regression",
)

number of parameters: 9.48M


In [ ]:
model